In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# Метод ближайших соседей

Задачи классификации и регрессии:  
$X - объекты, Y - ответы$  
$X^\ell = (x_i, y_i)^l_{i=1}$ - обучающая выборка

---
**Гипотеза компактности:** близкие объекты как правило лежат в одном классе.  
---
**Гипотеза непрерывности:** близким объектам как правило соответствуют близкие ответы.
---
---

**Формализация понятия "близости":  **

Задана функция расстояния $\rho:X\times X -> [0, \inf)$

*Пример - Евклидово расстояние:*  
    $\rho(x, x_i) = \left(\sum^{n}_{j=1}\left|x^j - x^j_i \right|^2\right)^{1/2}$

In [ ]:
from sklearn.neighbors import KNeighborsClassifier as kNN

In [ ]:
from sklearn import datasets
iris = datasets.load_iris()
X = iris.data[:, :2]
y = iris.target

In [ ]:
plt.figure(figsize=(10,10))
plt.scatter(X[:,0], X[:,1], c = y)
plt.show()

## Общая идея классификатора

Для каждого нового объекта $x$ отсортируем тренировочную выборку $(x_1,...,x_n)$ по степени удаления от него: 
$\rho(x,x^1) \le \rho(x,x^2)\le...  


**Метрический классификатор:**
---
$$a(x;X^\ell) = argmax_{y \in Y}\sum^{\ell}_{i=1}[y^i=y]w(i,x)$$

## Метод ближайшего соседа

$$w(i,x) = [i=1]$$  
Будем определять класс нового объекта по ближайшему примеру из обучающего множества.

И используем реализованный в sklearn алгоритм [KNeighborsClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html)

In [ ]:
clf = kNN(n_neighbors=1)
clf.fit(X,y)
print(clf)

In [ ]:
clf.predict([[4,4], [5,2], [10,5]])

In [ ]:
def get_grid(data, border = 1, step = 0.1):
    x_min, x_max = data[:,0].min()-border, data[:,0].max()+border
    y_min, y_max = data[:,1].min()-border, data[:,1].max()+border
    return np.meshgrid(np.arange(x_min,x_max, step),
                      np.arange(y_min, y_max, step))

In [ ]:
plt.figure(figsize=(10,10))
xx,yy = get_grid(X)
predicted = clf.predict(np.c_[xx.ravel(),yy.ravel()]).reshape(xx.shape)
plt.pcolormesh(xx,yy, predicted,cmap="bwr")
plt.scatter(X[:,0],X[:,1],c=y,s=100, cmap='bwr')
plt.show()

In [ ]:
clf = kNN(n_neighbors=3)

In [ ]:
clf.fit(X,y)

In [ ]:
np.argmax(clf.predict_proba([[5,1]]))

In [ ]:
def plot_model(X,y, clf):
    clf.fit(X,y)
    xx,yy = get_grid(X)
    predicted = clf.predict(np.c_[xx.ravel(),yy.ravel()]).reshape(xx.shape)
    plt.pcolormesh(xx,yy, predicted, cmap="bwr")
    plt.scatter(X[:,0],X[:,1],c=y,s=100, cmap="bwr")

Возможно дело в количестве учитываемых соседей?
Попробуем настроить параметр.

In [ ]:
for n_neighbors in [1,2,4,20]:
    plt.figure(figsize=(12,12))
    plot_model(X,y, kNN(n_neighbors=n_neighbors))

### Как выбрать оптимальное число соседей?

Разбиваем данные на 2 части - обучающую и контрольную выборки.  
<img src='holdout.png'/>

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, kNN(n_neighbors=3).fit(X_train, y_train).predict(X_test))

In [ ]:
n = 100
scores = []
for k in range(1, n):
    scores.append(accuracy_score(y_test, kNN(n_neighbors=k).fit(X_train, y_train).predict(X_test)))

In [ ]:
plt.figure(figsize=(18, 8))
plt.plot(range(1, n), scores)
plt.xlim(1, n)
plt.ylim(np.min(scores) - 0.1, 1)
plt.xticks(np.arange(1, n, 2))
plt.xlabel('Number of nearest neighbours')
plt.ylabel('Accuracy score')
plt.show()

**Какие параметры еще можно настроить?**  
1. Веса соседей(параметр weights объекта)
2. Метрику - metric

## Пример правильного обращения с признаками

Набор данных по классификации вин. Описание [тут](http://archive.ics.uci.edu/ml/datasets/Wine+Quality)

In [ ]:
data = pd.read_csv('http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv ' , sep = ';')

In [ ]:
X = data.drop('quality' , 1).values
y = data['quality'].values
pd.DataFrame.hist(data, figsize = [10,10]);

In [ ]:
data.head()

In [ ]:
y = data['quality'].values > 5
X = data.drop('quality', axis=1).values

<img src='cross_validation_diagram.png'/>

In [ ]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

def cv_nn(n_neighbors, X, y):
    average_scores = []    
    for k in n_neighbors:
        knn_clf = kNN(n_neighbors=k)
        scores = cross_val_score(knn_clf, X, y)
        average_scores.append(scores.mean())
    return average_scores

In [ ]:
n_neighbors = range(1, 51)
average_scores = cv_nn(n_neighbors, X, y)

In [ ]:
plt.figure(figsize=(18, 8))
l1 = plt.plot(n_neighbors, average_scores)
plt.xticks(n_neighbors)
plt.xlim(np.min(n_neighbors), np.max(n_neighbors))
plt.ylim(0, 1)
plt.xlabel('Number of nearest neighbours')
plt.ylabel('Accuracy score')
plt.show()

Отмасштабируем выборку с помощью функции scale:

In [ ]:
from sklearn.preprocessing import scale
X_scaled = scale(X)
average_scores_scaled = cv_nn(n_neighbors, X_scaled, y)

И сравним с немасштабированной:

In [ ]:
plt.figure(figsize=(18, 8))
plt.plot(n_neighbors, average_scores)
plt.plot(n_neighbors, average_scores_scaled)
plt.xticks(n_neighbors)
plt.xlim(np.min(n_neighbors), np.max(n_neighbors))
plt.ylim(0, 1)
plt.xlabel('Number of nearest neighbours')
plt.ylabel('Accuracy score')
plt.legend(['Unscaled data', 'Scaled data'], loc='lower right')
plt.show()